In [ ]:
!pip install transformers

다음 주관식 질문에 올바른 답을 작성하시오.

1. bert-base-uncased의 tokenizer를 이용해 "I love natural language processing"을 tokenize한 뒤 token, id, string을 반환하는 코드를 transformers 라이브러리를 이용해 작성하세요.

2. transformers의 pipeline class를 이용해 bert-base-uncased 모델을 load하고 "I [MASK] natural language processing"에서 "[MASK]"에 대한 예측값 상위 5개를 출력하는 코드를 작성하세요

3. bert-base-uncased의 tokenizer에서 1002번째 token의 string을 출력하는 코드를 작성하세요.

4. 다음 출력 문장과 정답 문장을 이용해 BLEU score를 계산하시오. (BLEU score 계산법은 강의자료를 따라주세요.) 

Brevity Score: 0.5  
Predicted Sentence: I like nlp but not you  
Reference: I love natural language processing how about you ?

다음 명제에 대해, True/False를 판단하시오. 판단 근거를 간략하게 설명하시오.

1. GPT-1,2,3 은 Bi-directional Transformer이다.

2. GPT-1은 여러가지의 task를 하나의 모델로 수행할 수 있다.

3. BERT는 classification을 하기 위해 encoder(transformer block)의 output을 모두 활용한다.

4. BERT의 [CLS] token은 반드시 첫번째 sequence에 위치 해야한다.

5. 두 문장의 의미가 서로 같은지 다른지 판단하는 task를 BERT를 이용하여 수행할 때 문장의 위치가 서로 바뀌더라도 성능은 동일하다.

### build_bpe 함수를 완성해주세요.

byte pair encoding을 이용한 간단한 sub-word tokenizer를 구현해봅니다.
과제 노트북의 지시사항과 각 함수의 docstring과 [논문](https://arxiv.org/pdf/1508.07909.pdf)의 3페이지 algorithm 1 참고하여 build_bpe 함수를 완성하고 모든 test case를 통과해주세요.

In [ ]:
from typing import List, Dict, Set
from itertools import chain
import re
from collections import defaultdict, Counter


def build_bpe(
        corpus: List[str],
        max_vocab_size: int
) -> List[int]:
    """ BPE Vocabulary Builder
    Implement vocabulary builder for byte pair encoding.
    Please sort your idx2word by subword length in descending manner.

    Hint: Counter in collection library would be helpful

    Note: If you convert sentences list to word frequence dictionary,
          building speed is enhanced significantly because duplicated words are
          preprocessed together

    Arguments:
    corpus -- List of words to build vocab
    max_vocab_size -- The maximum size of vocab

    Return:
    idx2word -- Subword list
    """
    # Special tokens
    PAD = BytePairEncoding.PAD_token  # Index of <PAD> must be 0
    UNK = BytePairEncoding.UNK_token  # Index of <UNK> must be 1
    CLS = BytePairEncoding.CLS_token  # Index of <CLS> must be 2
    SEP = BytePairEncoding.SEP_token  # Index of <SEP> must be 3
    MSK = BytePairEncoding.MSK_token  # Index of <MSK> must be 4
    SPECIAL = [PAD, UNK, CLS, SEP, MSK]

    WORD_END = BytePairEncoding.WORD_END  # Use this token as the end of a word
    # YOUR CODE HERE
    id2word = None
    
    return idx2word

In [ ]:
#############################################
# Helper functions below. DO NOT MODIFY!    #
#############################################

class BytePairEncoding(object):
    """ Byte Pair Encoding class
    We aren't gonna use this class for encoding. Because it is too slow......
    We will use sentence piece Google have made.
    Thus, this class is just for special token index reference.
    """
    PAD_token = '<pad>'
    PAD_token_idx = 0
    UNK_token = '<unk>'
    UNK_token_idx = 1
    CLS_token = '<cls>'
    CLS_token_idx = 2
    SEP_token = '<sep>'
    SEP_token_idx = 3
    MSK_token = '<msk>'
    MSK_token_idx = 4

    WORD_END = '_'

    def __init__(self, corpus: List[List[str]], max_vocab_size: int) -> None:
        self.idx2word = build_bpe(corpus, max_vocab_size)

    def encode(self, sentence: List[str]) -> List[int]:
        return encode(sentence, self.idx2word)

    def decoder(self, tokens: List[int]) -> List[str]:
        return decode(tokens, self.idx2word)


#############################################
# Testing functions below.                  #
#############################################


def test_build_bpe():
    print("======Building BPE Vocab Test Case======")
    PAD = BytePairEncoding.PAD_token
    UNK = BytePairEncoding.UNK_token
    CLS = BytePairEncoding.CLS_token
    SEP = BytePairEncoding.SEP_token
    MSK = BytePairEncoding.MSK_token
    WORD_END = BytePairEncoding.WORD_END

    # First test
    corpus = ['abcde']
    vocab = build_bpe(corpus, max_vocab_size=15)
    assert vocab[:5] == [PAD, UNK, CLS, SEP, MSK], \
        "Please insert the special tokens properly"
    print("The first test passed!")

    # Second test
    assert sorted(vocab[5:], key=len, reverse=True) == vocab[5:], \
        "Please sort your idx2word by subword length in decsending manner."
    print("The second test passed!")

    # Third test
    corpus = ['low'] * 5 + ['lower'] * 2 + ['newest'] * 6 + ['widest'] * 3
    vocab = set(build_bpe(corpus, max_vocab_size=24))
    assert vocab > {PAD, UNK, CLS, SEP, MSK, 'est_', 'low', 'newest_', \
                    'i', 'e', 'n', 't', 'd', 's', 'o', 'l', 'r', 'w',
                    WORD_END} and \
           "low_" not in vocab and "wi" not in vocab and "id" not in vocab, \
        "Your bpe result does not match expected result"
    print("The third test passed!")

    # forth test
    corpus = ['aaaaaaaaaaaa', 'abababab']
    vocab = set(build_bpe(corpus, max_vocab_size=13))
    assert vocab == {PAD, UNK, CLS, SEP, MSK, 'aaaaaaaa', 'aaaa', 'abab', 'aa',
                     'ab', 'a', 'b', WORD_END}, \
        "Your bpe result does not match expected result"
    print("The forth test passed!")

    # fifth test
    corpus = ['abc', 'bcd']
    vocab = build_bpe(corpus, max_vocab_size=10000)
    assert len(vocab) == 15, \
        "Your bpe result does not match expected result"
    print("The fifth test passed!")

    print("All 5 tests passed!")
test_build_bpe()